# 그냥 한번 뉴럴네트웤 테스트...ㅎ
* 데이터는 아래 데이터와, 그리고 각 사람당 리뷰한 맥주 데이터의 평균을 임베딩하여 추가함


In [2]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg
import numpy as np

from joblib import load, dump

### 데이터 설명
* review 테이블 : 사용자가 맥주에 준 점수, 즉 y값 하나만을 위해 가져옴
* beer 테이블 : 맥주 데이터
* member 테이블 : 멤버 나이, 성별
* member_mean 테이블 : 사용자의 현재까지 기록의 평균

In [3]:
conn = psycopg.connect(host="sulleong.site", dbname="sulleong", user="sulleong", password="Sulleong104**", port=5432)
sql = '''SELECT b.abv AS beer_abv, b.brewer_id, b.country, b.large_category, b.sub_category, m.age, m.gender, ms.*, r.overall AS y
    FROM member AS m
    INNER JOIN review AS r ON r.member_id = m.id
    INNER JOIN beer AS b ON r.beer_id = b.id
    INNER JOIN learning_dataset AS ms ON r.member_id = ms.member_id
    WHERE b.country IS NOT NULL;
    '''  # WHERE b.id IN (1, 2, 3);
reviews = sqlio.read_sql_query(sql, conn)
conn.close()

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10580\3287797993.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = sqlio.read_sql_query(sql, conn)


In [4]:
# 불러온 데이터, age랑 gender는 못써먹어서 일단 제외
reviews = reviews.drop(columns=['member_id', 'age', 'gender'])

# brewery가 너무 많아서 안될듯 ㅠㅠㅠ
reviews = reviews.drop(columns='brewer_id')

In [5]:
# 나누는것은 파이썬에서 해야하나...?ㅠㅠ
review_notdiv = reviews[['beer_abv', 'country', 'large_category', 'sub_category', 'y']]
review_todiv = reviews.drop(columns=['beer_abv', 'country', 'large_category', 'sub_category', 'y'])

In [6]:
review_todiv = review_todiv.div(review_todiv['divide_size'], axis=0)
review_todiv = review_todiv.drop(columns='divide_size')

In [7]:
reviews = pd.concat([review_notdiv, review_todiv], axis=1)

## 전처리
* 가져온 데이터 전처리 함
* 일단 무식하게 다 바꾸는 식으로 했는데
  * 이제 beer_id에 따라 범주화 후 그 데이터를 사용자리뷰와 join시키는 식으로 활용해야 할 듯

In [8]:
from sklearn.preprocessing import OneHotEncoder

# 인코더 설정, none값은 그냥 0 박음
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=int)

# 숫자형 데이터
numeric_data = reviews.drop(columns=['large_category', 'sub_category', 'country'])

# 범주형 데이터 맞추고 변환
encoder.fit(reviews[['large_category', 'sub_category', 'country']])
encoding_beer = encoder.transform(reviews[['large_category', 'sub_category', 'country']])
# # 다시 pandas 데이터프레임으로 변환해서 붙일수 있도록 만듬
encoded_categories_beer = pd.DataFrame(encoding_beer, columns=encoder.get_feature_names_out(input_features=['large_category', 'sub_category', 'country']))
# print(processed_beer_notscaled)
reviews = pd.concat([numeric_data, encoded_categories_beer], axis=1)

In [8]:
dump(encoder, './beer_encoder.joblib')

['./beer_encoder.joblib']

In [9]:
# abv에 null값 있음;;; 처리해야 함, 그냥 평균 넣음
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(reviews['beer_abv'].values.reshape(-1, 1))
reviews['beer_abv'] = imp.transform(reviews['beer_abv'].values.reshape(-1, 1))

In [10]:
y = reviews['y']
X = reviews.drop(columns=['y'])

In [11]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
dump(scaler, './model_scaler.joblib')

['./model_scaler.joblib']

In [18]:
np.where(np.isnan(X))

(array([], dtype=int64), array([], dtype=int64))

## 전처리가 모두 끝나면 regressor로 예측함

In [19]:
from sklearn.neural_network import MLPRegressor

# solver = ’adam’
# alpha = 0.0001
# batch_size = 200
# learning_rate = 0.001

regr = MLPRegressor(max_iter=250, hidden_layer_sizes=(50,50,30)).fit(X, y)

c:\Users\SSAFY\Desktop\realpjt\S09P22A104\recommend\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
# dump(regr, './model_regressor.joblib')
regr = load('./model_regressor.joblib')

['./model_regressor.joblib']

In [26]:
regre.predict(X[:15])

array([15.44460222, 14.0721475 , 15.40040175, 15.49860639, 17.04888018,
       15.64762113, 13.74225976, 11.29997593, 11.4592582 , 14.38282757,
       10.41384946, 14.49535821, 11.37318623, 13.47346987, 12.90725503])

In [21]:
y[:10]

0    13
1    15
2    11
3    15
4    18
5    14
6    14
7    13
8    13
9     6
Name: y, dtype: int64